In [1]:
import os # 修改工作目录

import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from jupyterquiz import display_quiz

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from scipy.stats import f
from scipy.stats import t

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import preprocessing

## 导入数据

In [2]:
Data = pd.read_csv("Project_3.csv")
print(Data.head())

    rpm  Road Octane Number  Compression  Brake Horsepower
0  2000                  90          100               225
1  1800                  94           95               212
2  2400                  88          110               229
3  1900                  91           96               222
4  1600                  86          100               219


## 参数设置

In [3]:
alpha = 0.05

# 数据规模
n = Data.shape[0]
p = Data.shape[1] - 1
print("The number of instances is ", n)
print("The number of features is ", p)

The number of instances is  12
The number of features is  3


### Task1 请用多元线性回归模型，描述制动马力和发动机转速、道路辛烷值以及压缩值之间的函数关系。

In [4]:
## Method 1: Matrix Calculus
Data1 = sm.add_constant(Data)
Data1_value = Data1.values
X = Data1_value[:,0:(p+1)]
y = Data1_value[:,-1]
beta_hat_1 = np.linalg.inv(X.T @ X) @ (X.T @ y)
# A @ B <=> np.dot(A,B) matrix multiply

print("The estimates of the parameters are \n", 
      np.around(beta_hat_1,4))

The estimates of the parameters are 
 [-2.660312e+02  1.070000e-02  3.134800e+00  1.867400e+00]
